<a href="https://colab.research.google.com/github/AyaSerageldin/Medical-Symptoms-Analysis/blob/master/YOLO_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade keras-cv
!pip install -q --upgrade keras  # Upgrade to Keras 3.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.9 MB/s eta 0:00:00


In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
!wget http://images.cocodataset.org/zips/test2017.zip
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip

--2024-10-30 20:11:27--  http://images.cocodataset.org/zips/train2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.69.156, 52.217.235.41, 52.217.117.57, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.69.156|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19336861798 (18G) [application/zip]
Saving to: ‘train2017.zip’

train2017.zip       100%[===================>]  18.01G  52.0MB/s    in 7m 26s  

2024-10-30 20:18:53 (41.4 MB/s) - ‘train2017.zip’ saved [19336861798/19336861798]

--2024-10-30 20:18:53--  http://images.cocodataset.org/zips/val2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 16.182.38.233, 3.5.28.154, 3.5.29.149, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|16.182.38.233|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 815585330 (778M) [application/zip]
Saving to: ‘val2017.zip’

val2017.zip         100%[===================>] 777.80

In [ ]:
import zipfile
import os

# Unzipping train, val, and test datasets
for zip_file in ['train2017.zip', 'val2017.zip', 'test2017.zip', 'annotations_trainval2017.zip']:
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall()

In [ ]:
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
os.makedirs('data/test', exist_ok=True)

# Move images to respective directories
!mv train2017/* data/train/
!mv val2017/* data/val/
!mv test2017/* data/test/

/bin/bash: line 1: /usr/bin/mv: Argument list too long


In [ ]:
import os

# Create main data directory
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
os.makedirs('data/annotations', exist_ok=True)


# Move training images
!mv train2017/* data/train/

# Move validation images
!mv val2017/* data/val/

# Move the annotations JSON file into the annotations directory
!mv annotations/* data/annotations/

/bin/bash: line 1: /usr/bin/mv: Argument list too long
mv: cannot stat 'val2017/*': No such file or directory


In [ ]:
!unzip val2017.zip -d val2017



Streaming output truncated to the last 5000 lines.
 extracting: val2017/val2017/000000212226.jpg  
 extracting: val2017/val2017/000000231527.jpg  
 extracting: val2017/val2017/000000578922.jpg  
 extracting: val2017/val2017/000000062808.jpg  
 extracting: val2017/val2017/000000119038.jpg  
 extracting: val2017/val2017/000000114871.jpg  
 extracting: val2017/val2017/000000463918.jpg  
 extracting: val2017/val2017/000000365745.jpg  
 extracting: val2017/val2017/000000320425.jpg  
 extracting: val2017/val2017/000000481404.jpg  
 extracting: val2017/val2017/000000314294.jpg  
 extracting: val2017/val2017/000000335328.jpg  
 extracting: val2017/val2017/000000513688.jpg  
 extracting: val2017/val2017/000000158548.jpg  
 extracting: val2017/val2017/000000132116.jpg  
 extracting: val2017/val2017/000000415238.jpg  
 extracting: val2017/val2017/000000321333.jpg  
 extracting: val2017/val2017/000000081738.jpg  
 extracting: val2017/val2017/000000577584.jpg  
 extracting: val2017/val2017/00000034

In [ ]:
!ls data/annotations

annotations		 instances_train2017.json	  person_keypoints_val2017.json
captions_train2017.json  instances_val2017.json
captions_val2017.json	 person_keypoints_train2017.json


In [ ]:
!pip install pycocotools

In [ ]:
import os
import json
from pycocotools.coco import COCO

# Paths
train_annotations_path = 'data/annotations/instances_train2017.json'
val_annotations_path = 'data/annotations/instances_val2017.json'
train_images_dir = 'data/train'
val_images_dir = 'data/val'

# Load the COCO annotations
coco_train = COCO(train_annotations_path)
coco_val = COCO(val_annotations_path)

# Function to convert COCO annotations to YOLO format
def convert_coco_to_yolo(coco, images_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_id in coco.imgs.keys():
        img_info = coco.imgs[img_id]
        # Get image dimensions
        width, height = img_info['width'], img_info['height']
        # Get image file name
        img_file_name = img_info['file_name']

        # Create a corresponding text file for annotations
        yolo_annotations = []

        # Get all annotation IDs for the image
        ann_ids = coco.getAnnIds(imgIds=img_id)
        annotations = coco.loadAnns(ann_ids)

        for ann in annotations:
            # COCO format (x, y, width, height)
            x, y, w, h = ann['bbox']
            # Convert to YOLO format (center_x, center_y, width, height)
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            w_yolo = w / width
            h_yolo = h / height
            category_id = ann['category_id']  # Class ID

            yolo_annotations.append(f"{category_id} {x_center} {y_center} {w_yolo} {h_yolo}")

        # Save annotations to a .txt file
        txt_file_path = os.path.join(output_dir, img_file_name.replace('.jpg', '.txt'))  # Change this based on your image format
        with open(txt_file_path, 'w') as f:
            for line in yolo_annotations:
                f.write(line + '\n')

# Convert training and validation annotations
convert_coco_to_yolo(coco_train, train_images_dir, 'data/train_labels')
convert_coco_to_yolo(coco_val, val_images_dir, 'data/val_labels')

loading annotations into memory...
Done (t=27.30s)
creating index...
index created!
loading annotations into memory...
Done (t=5.30s)
creating index...
index created!


In [ ]:
!ls data/train_labels
!ls data/val_labels

Streaming output truncated to the last 5000 lines.
000000096586.txt  000000213381.txt  000000329715.txt  000000445603.txt	000000562507.txt
000000096589.txt  000000213382.txt  000000329717.txt  000000445605.txt	000000562510.txt
000000096596.txt  000000213389.txt  000000329724.txt  000000445606.txt	000000562516.txt
000000096615.txt  000000213391.txt  000000329726.txt  000000445607.txt	000000562517.txt
000000096618.txt  000000213393.txt  000000329730.txt  000000445620.txt	000000562519.txt
000000096622.txt  000000213399.txt  000000329733.txt  000000445621.txt	000000562523.txt
000000096625.txt  000000213403.txt  000000329734.txt  000000445628.txt	000000562527.txt
000000096628.txt  000000213405.txt  000000329737.txt  000000445638.txt	000000562544.txt
000000096630.txt  000000213408.txt  000000329738.txt  000000445639.txt	000000562556.txt
000000096633.txt  000000213412.txt  000000329748.txt  000000445643.txt	000000562557.txt
000000096638.txt  000000213419.txt  000000329751.txt  000000445662.tx

In [ ]:
!pip install keras-cv

In [ ]:
!pip install keras-cv tensorflow

In [ ]:
!pip install keras-cv tensorflow pycocotools

In [ ]:
import keras_cv
import os
import tensorflow as tf
from keras_cv.models import YOLOV8Detector

In [ ]:
!pip install tensorflow

In [ ]:
import os
import tensorflow as tf

# Function to parse and load images and labels
def parse_function(image_path, label_path):
    # Load and preprocess the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [640, 640])  # Resize to input shape
    image = image / 255.0  # Normalize to [0, 1]

    # Load the label
    label = tf.io.read_file(label_path)
    label = tf.strings.split(label, sep="\n")[:-1]  # Split and remove the last empty string

    # Initialize a TensorArray to store bounding box data
    boxes = tf.TensorArray(tf.float32, size=0, dynamic_size=True)

    for obj in label:
        if tf.strings.length(obj) > 0:
            parts = tf.strings.split(obj, sep=" ")
            class_id = tf.strings.to_number(parts[0], out_type=tf.float32)  # Change to float32
            x_center = tf.strings.to_number(parts[1], out_type=tf.float32)  # x_center
            y_center = tf.strings.to_number(parts[2], out_type=tf.float32)  # y_center
            width = tf.strings.to_number(parts[3], out_type=tf.float32)  # width
            height = tf.strings.to_number(parts[4], out_type=tf.float32)  # height

            # Stack bounding box data and write it to the TensorArray
            boxes = boxes.write(boxes.size(), tf.stack([class_id, x_center, y_center, width, height]))

    # Convert the TensorArray to a tensor after finishing the loop
    boxes = boxes.stack() if boxes.size() > 0 else tf.zeros((0, 5), dtype=tf.float32)  # Return an empty tensor if no boxes

    return image, boxes  # Return image and formatted boxes

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_dataset = val_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

# Make sure to inspect the structure of your dataset
for images, boxes in train_dataset.take(1):
    print(f"Image shape: {images.shape}")
    print(f"Boxes shape: {boxes.shape}")

In [ ]:
for images, boxes in train_dataset.take(1):
    print(f"Image shape: {images.shape}")
    print(f"Boxes shape: {boxes.shape}")

In [ ]:
# import os
# import tensorflow as tf

# # Function to load YOLO format dataset
# def load_yolo_dataset(images_dir, labels_dir):
#     image_paths = []
#     label_paths = []

#     # Collect all image and label file paths
#     for filename in os.listdir(images_dir):
#         if filename.endswith('.jpg'):  # Change this based on your image format
#             image_paths.append(os.path.join(images_dir, filename))
#             label_paths.append(os.path.join(labels_dir, filename.replace('.jpg', '.txt')))

#     return image_paths, label_paths

# train_images_dir = 'data/train'
# train_labels_dir = 'data/train_labels'
# val_images_dir = 'data/val'
# val_labels_dir = 'data/val_labels'

# # Load training and validation dataset paths
# train_images, train_labels = load_yolo_dataset(train_images_dir, train_labels_dir)
# val_images, val_labels = load_yolo_dataset(val_images_dir, val_labels_dir)

# # Convert lists of file paths to TensorFlow tensor strings
# train_images = tf.convert_to_tensor(train_images, dtype=tf.string)
# train_labels = tf.convert_to_tensor(train_labels, dtype=tf.string)
# val_images = tf.convert_to_tensor(val_images, dtype=tf.string)
# val_labels = tf.convert_to_tensor(val_labels, dtype=tf.string)

# # Function to parse and load images and labels
# def parse_function(image_path, label_path):
#     # Load and preprocess the image
#     image = tf.io.read_file(image_path)
#     image = tf.image.decode_jpeg(image, channels=3)
#     image = tf.image.resize(image, [640, 640])  # Resize to input shape
#     image = image / 255.0  # Normalize to [0, 1]

#     # Load the label
#    label = tf.io.read_file(label_path)
#   label = tf.strings.split(label, sep="\n")[:-1]  # Split and remove the last empty string
#     Convert label strings to float
#     label = tf.strings.to_number(label, out_type=tf.float32)

#     return image, label




# # Create TensorFlow datasets
# train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
# train_dataset = train_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
# train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

# val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
# val_dataset = val_dataset.map(parse_function, num_parallel_calls=tf.data.AUTOTUNE)
# val_dataset = val_dataset.batch(16).prefetch(tf.data.AUTOTUNE)
# # At this point, you can continue with the YOLOv8 training steps as before.



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 41)

In [ ]:
!pip install --upgrade keras-cv

In [ ]:
import os

# Function to extract unique class indices from label files
def get_unique_classes(labels_dir):
    class_indices = set()  # Use a set to store unique class indices

    for filename in os.listdir(labels_dir):
        if filename.endswith('.txt'):  # Check if the file is a label file
            label_path = os.path.join(labels_dir, filename)
            with open(label_path, 'r') as file:
                for line in file:
                    # Each line contains the class index and bounding box coordinates
                    parts = line.split()
                    if parts:  # Ensure the line is not empty
                        class_index = int(parts[0])  # Get the class index
                        class_indices.add(class_index)  # Add it to the set

    return sorted(class_indices)  # Return sorted list of unique classes

# Assuming your train and val labels are in these directories
train_labels_dir = 'data/train_labels'
val_labels_dir = 'data/val_labels'

# Extract unique class indices from both training and validation sets
unique_classes_train = get_unique_classes(train_labels_dir)
unique_classes_val = get_unique_classes(val_labels_dir)

# Combine unique classes from both sets
unique_classes = sorted(set(unique_classes_train).union(set(unique_classes_val)))
num_classes = len(unique_classes)  # Count of unique classes

print("Unique classes:", unique_classes)
print("Number of classes:", num_classes)

Unique classes: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]
Number of classes: 80


In [ ]:
from keras_cv.models import YOLOV8Detector, YOLOV8Backbone

# Choose your backbone model
backbone = YOLOV8Backbone.from_preset("yolo_v8_s_backbone_coco")  # Change as needed

# Create the YOLOv8 Detector
yolo_model = YOLOV8Detector(
    backbone=backbone,  # Use the backbone we just created
    num_classes=80, # Use the number of unique classes found
    bounding_box_format="xyxy",  # Adjust based on your label format
    fpn_depth=1,  # Adjust if necessary
)



In [ ]:
from keras_cv.models import YOLOV8Detector, YOLOV8Backbone
from keras.losses import MeanSquaredError, SparseCategoricalCrossentropy

# Define the loss functions
box_loss = MeanSquaredError()  # Use an appropriate box loss function (you can use other options)
classification_loss = SparseCategoricalCrossentropy()  # Use a suitable classification loss

# Compile the model
yolo_model.compile(
    optimizer='adam',
    box_loss=box_loss,
    classification_loss=classification_loss
)



In [ ]:
# Now you can proceed to train the model
yolo_model.fit(train_dataset, validation_data=val_dataset, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor_2392']. Received: the structure of inputs=*
  warnings.warn(


TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got 'classes'